In [ ]:
from decimal import Decimal
import math
from typing import Tuple

import pandas_ta as ta

from hummingbot.core.data_type.common import PriceType, TradeType
from hummingbot.data_feed.candles_feed.data_types import CandlesConfig
from controllers.market_making.pmm_simple import PMMSimpleConfig, PMMSimpleController

from pydantic import Field

class ASMMControllerConfig(PMMSimpleConfig):
  # идентификатор контроллера для YAML и загрузчика
    controller_name: str = "asmm_controller"

    # параметры A&S
    gamma: float = Field(0.5, json_schema_extra={
        "prompt_on_new": True,
        "prompt": "Gamma (risk aversion): ",
        "is_updatable": True
    })
    kappa: float = Field(1.0, json_schema_extra={
        "prompt_on_new": True,
        "prompt": "Kappa (order arrival rate): ",
        "is_updatable": True
    })

    # параметры данных свечей
    candles_connector_name: str = Field(default="binance")
    candles_interval: str = Field(default="1m")
    candles_length: int = Field(default=30)


class ASMMController(PMMSimpleController):
    """
    PMM Simple + Avellaneda–Stoikov ценообразование:
      reservation_price = mid - inv_dev * gamma * sigma^2
      optimal_spread   = gamma * sigma^2 + (2 / gamma) * ln(1 + gamma / kappa)
    """
    def __init__(self, config: ASMMControllerConfig, *args, **kwargs):
        # Инициализируем свечи через config — их поднимет MarketDataProvider
        if len(config.candles_config) == 0:
            config.candles_config = [
                CandlesConfig(
                    connector=config.candles_connector_name,
                    trading_pair=config.trading_pair,
                    interval=config.candles_interval
                )
            ]
        super().__init__(config, *args, **kwargs)
        self.config = config

    async def update_processed_data(self):
        # 1) Базовая цена: mid
        mid_price = Decimal(
            self.market_data_provider.get_price_by_type(
                self.config.connector_name, self.config.trading_pair, PriceType.MidPrice
            )
        )

        # 2) Волатильность: sigma из NATR (как в PMMDynamic)
        candles = self.market_data_provider.get_candles_df(
            connector_name=self.config.candles_connector_name,
            trading_pair=self.config.trading_pair,
            interval=self.config.candles_interval,
            max_records=self.config.candles_length
        )
        if candles is None or candles.empty or len(candles) < self.config.candles_length:
            sigma = Decimal("0.01")  # запасной вариант
        else:
            natr = ta.natr(candles["high"], candles["low"], candles["close"], length=self.config.candles_length) / 100
            sigma = Decimal(str(natr.iloc[-1]))

        # 3) Инвентарь: нормируем текущую базовую позицию к общей quote-аллокции
        #    q_quote = q_base * mid; inv_dev = q_quote / total_amount_quote  in [-1, 1] примерно
        total_amount_quote = Decimal(str(self.config.total_amount_quote))
        if total_amount_quote and total_amount_quote > 0:
            q_base = self.get_current_base_position()
            inv_dev = (q_base * mid_price) / total_amount_quote
        else:
            inv_dev = Decimal("0")

        gamma = Decimal(str(self.config.gamma))
        kappa = Decimal(str(self.config.kappa))

        # 4) Формулы A&S
        reservation_price = mid_price - inv_dev * gamma * (sigma ** 2)
        # ln берем из math.log по float, потом обратно в Decimal
        ln_term = Decimal(str(math.log(1.0 + float(gamma / kappa)))) if kappa != 0 else Decimal("0")
        optimal_spread = gamma * (sigma ** 2) + (Decimal("2") / gamma) * ln_term

        # Сохраняем метрики для статуса/дебага
        self.processed_data = {
            "mid_price": mid_price,
            "reservation_price": reservation_price,
            "optimal_spread": optimal_spread,
            "sigma": sigma,
        }

    def get_price_and_amount(self, level_id: str) -> Tuple[Decimal, Decimal]:
        """
        Заменяем ценовую модель уровней: вместо конфигных спредов используем A&S.
        Объемы по уровням берём из конфигурации (как раньше).
        """
        level = self.get_level_from_level_id(level_id)
        trade_type = self.get_trade_type_from_level_id(level_id)

        reservation_price: Decimal = self.processed_data["reservation_price"]
        optimal_spread: Decimal = self.processed_data["optimal_spread"]

        # Простая лестница по спреду: 1x, 2x, 3x...
        spread_for_level = optimal_spread * Decimal(level + 1)
        side_multiplier = Decimal("-1") if trade_type == TradeType.BUY else Decimal("1")
        order_price = reservation_price * (Decimal("1") + side_multiplier * (spread_for_level / Decimal("2")))

        # Объем: берём распределение по уровням из конфига, конвертируем в base
        _, amounts_quote = self.config.get_spreads_and_amounts_in_quote(trade_type)
        amount_base = Decimal(str(amounts_quote[level])) / order_price

        return order_price, amount_base